# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,crane,31.3974,-102.3501,19.47,11,3,2.72,US,1712545631
1,1,bonthe,7.5264,-12.5050,25.82,86,70,2.03,SL,1712545798
2,2,puerto natales,-51.7236,-72.4875,2.97,73,100,3.38,CL,1712545700
3,3,san antonio de pale,-1.4014,5.6325,28.24,79,91,4.33,GQ,1712545800
4,4,hermanus,-34.4187,19.2345,17.11,94,100,9.72,ZA,1712545641


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [28]:
%%capture --no-display

# Configure the map plot
cities_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City"
)

# Display the map
cities_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [16]:
# Narrow down cities that fit criteria and drop any results with null values
# Cities with wind speed less than equal to 0.5 m/s
ideal_df = city_data_df.loc[city_data_df["Wind Speed"] <= 0.50]

# Drop any rows with null values
ideal_df.dropna()

# Display sample data
ideal_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
21,21,blackmans bay,-43.0167,147.3167,17.07,52,95,0.45,AU,1712545754
50,50,barentu,15.1139,37.5928,23.94,59,66,0.42,ER,1712545821
145,145,diego de almagro,-26.3667,-70.0500,17.16,50,0,0.12,CL,1712545862
150,150,valparaiso,-33.0393,-71.6273,11.80,87,0,0.00,CL,1712545857
210,210,bethel,41.3712,-73.4140,6.45,63,0,0.00,US,1712545725
222,222,songea,-10.6833,35.6500,17.32,97,100,0.36,TZ,1712545898
233,233,canutama,-6.5339,-64.3831,22.89,99,48,0.43,BR,1712545902
267,267,greenland,43.0362,-70.8328,3.81,86,0,0.00,US,1712545917
311,311,donaghadee,54.6333,-5.5333,5.96,90,30,0.45,GB,1712545930
327,327,cururupu,-1.8283,-44.8683,22.72,100,100,0.14,BR,1712545947


### Step 3: Create a new DataFrame called `hotel_df`.

In [17]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df.copy()
hotel_df = hotel_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
21,blackmans bay,AU,-43.0167,147.3167,52,
50,barentu,ER,15.1139,37.5928,59,
145,diego de almagro,CL,-26.3667,-70.0500,50,
150,valparaiso,CL,-33.0393,-71.6273,87,
210,bethel,US,41.3712,-73.4140,63,
222,songea,TZ,-10.6833,35.6500,97,
233,canutama,BR,-6.5339,-64.3831,99,
267,greenland,US,43.0362,-70.8328,86,
311,donaghadee,GB,54.6333,-5.5333,90,
327,cururupu,BR,-1.8283,-44.8683,100,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [18]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel",
          "apiKey": geoapify_key,
          "limit": 20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
blackmans bay - nearest hotel: Villa Howden
barentu - nearest hotel: No hotel found
diego de almagro - nearest hotel: Residencial Hambyll
valparaiso - nearest hotel: Ibis Hotel
bethel - nearest hotel: Hampton Inn Danbury
songea - nearest hotel: No hotel found
canutama - nearest hotel: No hotel found
greenland - nearest hotel: The Inn at Rye Place
donaghadee - nearest hotel: Premier Inn
cururupu - nearest hotel: No hotel found
sale - nearest hotel: Le dawliz
kill devil hills - nearest hotel: Mariner Days Inn & Suites
roma - nearest hotel: Wingate by Wyndham Rome
aconibe - nearest hotel: No hotel found
pekanbaru - nearest hotel: Hotel Radja
mbuji-mayi - nearest hotel: Hôtel Gloria
fort bragg - nearest hotel: Airborne Inn Lodging
bandar labuan - nearest hotel: Tiara Labuan Hotel
aripuana - nearest hotel: No hotel found
la romana - nearest hotel: Hotel Frano
kunduz - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
21,blackmans bay,AU,-43.0167,147.3167,52,Villa Howden
50,barentu,ER,15.1139,37.5928,59,No hotel found
145,diego de almagro,CL,-26.3667,-70.0500,50,Residencial Hambyll
150,valparaiso,CL,-33.0393,-71.6273,87,Ibis Hotel
210,bethel,US,41.3712,-73.4140,63,Hampton Inn Danbury
222,songea,TZ,-10.6833,35.6500,97,No hotel found
233,canutama,BR,-6.5339,-64.3831,99,No hotel found
267,greenland,US,43.0362,-70.8328,86,The Inn at Rye Place
311,donaghadee,GB,54.6333,-5.5333,90,Premier Inn
327,cururupu,BR,-1.8283,-44.8683,100,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [29]:
%%capture --no-display

# Configure the map plot
hotels_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotels_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)